In [ ]:
from pathlib import Path
from freqtrade.configuration import Configuration

In [ ]:
config = Configuration.from_files(["../../config.json"])
config["timeframe"] = "1m"
config["strategy"] = "MyStrategy"
pair = config["pairs"][0]
pair

In [ ]:
from re import A
from freqtrade.data.history import load_pair_history
from freqtrade.enums import CandleType

data_location = Path(config["user_data_dir"], "data", "binance")
candles = load_pair_history(
    datadir=data_location,
    timeframe=config["timeframe"],
    pair=pair,
    data_format="hdf5",
    candle_type=CandleType.FUTURES,
)
print(f"Loaded {str(len(candles))} rows of data for {pair} from {data_location}")
candles

In [ ]:
from freqtrade.resolvers import StrategyResolver
from freqtrade.data.dataprovider import DataProvider
strategy = StrategyResolver.load_strategy(config)
strategy.dp = DataProvider(config, None, None)

# Generate buy/sell signals using strategy
df = strategy.analyze_ticker(candles, {"pair": pair})
df.tail()

In [ ]:
import numpy as np

np.nanmin((df["rsi"]).values)

In [ ]:
df.loc[df["enter_long"]==1]

In [ ]:
# Report results
print(f"Generated {df['enter_long'].sum()} entry signals")
data = df.set_index('date', drop=False)

## Backtest results analysis

In [ ]:
from freqtrade.data.btanalysis import load_backtest_data, load_backtest_stats

backtest_file = config["user_data_dir"] / "backtest_results" / "backtest-result-2022-08-02_02-45-14.json"

stats = load_backtest_stats(backtest_file)

In [ ]:
stats["strategy_comparison"]

In [ ]:
# Load backtested trades as dataframe
trades = load_backtest_data(backtest_file)

# Show value-counts per pair
trades.groupby("pair")["exit_reason"].value_counts()

## Plot backtest results

In [ ]:
from freqtrade.configuration import Configuration
from freqtrade.data.btanalysis import load_backtest_data, load_backtest_stats
import plotly.express as px
import pandas as pd

stats = load_backtest_stats(backtest_file)
strategy_stats = stats['strategy']["MyStrategy"]

dates = []
profits = []
for date_profit in strategy_stats['daily_profit']:
    dates.append(date_profit[0])
    profits.append(date_profit[1])


equity = 0
equity_daily = []
for daily_profit in profits:
    equity_daily.append(equity)
    equity += float(daily_profit)


df = pd.DataFrame({'dates': dates, 'equity_daily': equity_daily})

fig = px.line(df, x="dates", y="equity_daily")
fig.show()

In [ ]:
from freqtrade.plot.plotting import  generate_candlestick_graph
# Limit graph period to keep plotly quick and reactive

# Filter trades to one pair
trades_red = trades.loc[trades['pair'] == pair]

data_red = data['2022-07-02':'2022-08-01']
# Generate candlestick graph
graph = generate_candlestick_graph(pair=pair,
                                   data=data_red,
                                   trades=trades_red,
                                   indicators1=['sma20', 'sma60'],
                                   # indicators2=['rsi', 'macd', 'macdsignal', 'macdhist']
                                  )
# Render graph in a seperate window
graph.show(renderer="browser")

In [ ]:
trades['profit_ratio']

In [ ]:
import plotly.figure_factory as ff

hist_data = [trades.profit_ratio]
group_labels = ['profit_ratio']  # name of the dataset

fig = ff.create_distplot(hist_data, group_labels, bin_size=0.001)
fig.show()